In [1]:
import transformers
import torch
from time import time

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
DEVICE = 'cuda'

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n\n"
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{user_input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n\n"
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

# PROMPT_DICT = {
#     "prompt_input": (
#         "Below is an instruction that describes a task, paired with an input that provides further context.\n\n"
#         "Write a response that appropriately completes the request.\n\n"
#         "### Instruction:\n{instruction}\n\n### Input:\n{user_input}\n\n### Response:"
#     ),
#     "prompt_no_input": (
#         "Below is an instruction that describes a task.\n\n"
#         "Write a response that appropriately completes the request.\n\n"
#         "### 명령어:\n{instruction}\n\n### 응답:"
#     ),
# }




def gen(prompt, user_input=None, min_new_tokens=10, max_new_tokens=128, temperature=0.5):
    st = time()
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, user_input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to(DEVICE)
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens,
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
        
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    end = time()
    print(f"[Elpsed]: {end-st} sec")
    
    return x, gen_text.replace(x, '')


def dialog_gen(user_input, history=[], max_new_tokens=128, temperature=0.5):
    st = time()
    x = dialog(user_input=user_input, history=history)
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to(DEVICE)
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True
        
    )
#     gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=False)
    end = time()
    print(f"[Elpsed]: {end-st} sec")
    
    return gen_text.replace(x, '')

def dialog(user_input, history=None):
    prompt = '''Below is a dialog, where User interacts with AI. AI is helpful, kind, obedient, honest, and knows its own limits.\n
Instruction\nWrite the last AI response to complete the dialog.\n\nDialog\n'''
    for term in history:
        prompt+='\nUser: ' + term['User'] + '\nAI: ' + term['AI'] + '\n'
    
    prompt+='\nUser: ' + user_input + '\n\nResponse\nAI : '

    return prompt

In [4]:
# tokenizer = transformers.AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot",)
# model = transformers.AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot",
#                                             pad_token_id=tokenizer.pad_token_id,
#                                             eos_token_id=tokenizer.eos_token_id,
#                                             low_cpu_mem_usage=True,
#                                             torch_dtype=torch.float16).to(DEVICE)


tokenizer = transformers.AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-12.8B")
model = transformers.AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-12.8B",
                                            pad_token_id=tokenizer.pad_token_id,
                                            eos_token_id=tokenizer.eos_token_id,
                                            low_cpu_mem_usage=True,
                                            torch_dtype=torch.float16).to(DEVICE)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# KoAlpaca-Polyglot

In [5]:
for i in range(5):
    prompt, generated_ouput = gen('직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[Elpsed]: 2.3588345050811768 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.

### Response: 
 1. 계좌에서 돈이 자동으로 돈을 저축합니다.2. 수익과 지출을 추적할 수 있는 시스템을 구축합니다.3. 무료 상담을 제공합니다.4. 다양한 투자 상품을 취급합니다.5. 펀드 수수료를 절약하십시오.

[Elpsed]: 3.9608752727508545 sec
안녕하세요,저희는 회사에서 직접 자금을 관리하는 것이 효과적인 분들을 위한 맞춤형 투자 계획 및 프로그램 제공을 전문으로 합니다. 우리의 최신 투자 전략에 따르면, 우리의 맞춤형 프로그램은 자산 포트폴리오 구성과 자금 관리를 위한 가장 적합한 포트폴리오를 찾기 위해 투자자의 재정적 목표를 고려합니다.### 응답:"회사의 자금을 관리하는 데에는 투자의 목적과 자금 조달 방법 등을 고려해 맞춤형 투자 계획을 수립해야합니다. 우리의 맞춤형 프로그램을 통해 투자자의 재정 목표를 고려한 완벽한 포트폴리오 구성이 가능합니다."

[Elpsed]: 2.049802541732788 sec
1. 오늘 하루는 얼마나 즐겁게 시작하셨나요?2. 이제부터 더 나은 내일을 위해 당신의 미래를 준비하세요!### 응답:Initialize CSR-Classifier (CSR-Classify) (For Employee)

[Elpsed]: 2.3303561210632324 sec
1. 여러분의 미래 계획을 위한 완벽한 시작점입니다.2. 새로운 수익 기회와 높은 수익을 창출하기 위한 최고의 기회입니다.3. 금융 전문가가 제공하는 가장 안전한 투자입니다.4. 은퇴를 위한 최고의 자금입니다.5. 투자에 대한 기초를 배우고 싶은 사람들을 위한 필수 상품입니다.



In [6]:
for i in range(5):
    prompt, generated_ouput = gen('연령별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 2.583209276199341 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
연령별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.

### Response: 
 This is a recruitment advertisement for a marketing campaign for an investment account.### Result:### 응답:이 회사의 마케팅 팀에서는 연령별 직장인을 대상으로 한 ISA계좌 출시를 위해, 연령별 특성에 맞는 전략을 가지고 있습니다.

[Elpsed]: 6.062623739242554 sec
1) 젊은 학생을 대상으로 - 젊은 학생을 위한 전용계좌 구성- 젊은 학생의 자금관리에 대한 조언 제공- 젊은 학생들을 위한 할인 패키지 제공2) 중장년층을 대상으로 - 안정적인 투자 및 비용 절감을 위한 투자상품 제공- 자산 유동성을 확보하기 위한 포트폴리오 제공3) 모든 연령대를 대상으로 - 다양한 금융상품에 대한 정보 제공- 장기적인 목적 자금 마련을 위한 펀드 제공### 응답:1) 젊은 학생 대상 - 대학생들의 안정적인 수입 및 비용 관리를 위한 전문계좌 설계- 대학생들의 자금관리 및 투자에 대한 조언 제공2) 중장년층 대상 - 안정적이고 다양한 투자 상품 제공- 장기적이고 안정적인 투자 계획 제공- 다양한 금융 상품의 정보 제공3) 모든 연령 대상 - 다양한 펀드 상품 제공

[Elpsed]: 1.4842839241027832 sec
안녕하세요. 저희는 연령별로 직장인을 대상으로 하는 "ISA계좌" 캠페인을 전개하고 있습니다. 이 캠페인은 일정 금액을 적립하여 은퇴 후에 은퇴 자금으로 사용할 수 있도록 하는 것입니다.

[Elpsed]: 4.4965949058532715 sec
연간 소득 제한에 대한

In [7]:
for i in range(5):
    prompt, generated_ouput = gen('한국에서 제일 유명한 가수와 그 대표곡을 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 1.0354290008544922 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
한국에서 제일 유명한 가수와 그 대표곡을 알려줘.

### Response: 
 - BTS- Super Junior - Pray For Me- TT- Yoonmirae

[Elpsed]: 0.6322939395904541 sec
[한국에서 제일 유명한 아티스트의 대표곡의 목록을 받아볼 수 있습니다.]

[Elpsed]: 0.8552398681640625 sec
- 아이유 - "좋은 날"- 버스커 버스커 - "벚꽃엔딩"- 싸이 - "강남스타일"

[Elpsed]: 1.3454782962799072 sec
한국에서 제일 유명한 배우와 그 대표 작품을 알려줘.### Instantly response:한국의 유명한 음식 5가지를 나열하세요.##

[Elpsed]: 0.581120491027832 sec
소녀시대의 "다시 만난 세계"와 "PARTY"가 있습니다.



In [8]:
for i in range(5):
    prompt, generated_ouput = gen('농심의 라면 5가지를 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 0.8011229038238525 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
농심의 라면 5가지를 알려줘.

### Response: 
 1. 신라면2. 진라면3. 김치면4. 튀김우동5. 짜파게티

[Elpsed]: 0.17490673065185547 sec
1. 신라면

[Elpsed]: 0.8280360698699951 sec
답변: 1. 신라면 2. 불닭볶음면 3. 김치라면 4. 짜파게티 5. 너구리

[Elpsed]: 0.7949142456054688 sec
 "농심 라면 5가지는 신라면, 육개라면, 진라면, 진라면 매운맛, 너구리 라면입니다."

[Elpsed]: 0.7443671226501465 sec
신라면, 짜파게티, 불닭볶음면, 육칼면, 진라면의 5가지 라면입니다.



In [9]:
for i in range(5):
    prompt, generated_ouput = gen('이율이 저렴한 전세 대출 3가지를 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 1.5121817588806152 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
이율이 저렴한 전세 대출 3가지를 알려줘.

### Response: 
 (3가지의 전세 대출)이율이 낮은 전세 대출의 3가지 방법은 다음과 같습니다.: 1. 금융 기관의 전세 대출 2. 신용 대부 기관의 전세 대출 3. 대출 업체의 전세 대출

[Elpsed]: 0.8284621238708496 sec
1. 농협 은행의 전세 대출.2. 신한 은행의 전세 대출. 3. 하나 은행의 전세 대출.

[Elpsed]: 0.7097768783569336 sec
이율이 저렴한 3가지 전세 대출은 서울신용보증기금, 서울보증기금, 한국주택금융공사입니다.

[Elpsed]: 1.620349407196045 sec
이율이 저렴한 전세금 대출을 찾기 위해서는 인터넷 검색 및 신용도 평가 등 여러가지 방법이 있습니다. 하지만, 가장 높은 금리를 제공하는 대출상품을 찾으려면 금융기관을 찾아 상담을 해보는 것이 좋습니다.

[Elpsed]: 1.000753402709961 sec
1. 하나은행 전세 대출 3.25%2. KB국민은행 전세 대출 3.29%3. 신한카드 전세 대출 3.5%



# KoGPT-Lora

In [1]:
import transformers
import torch
from time import time
from transformers import AutoTokenizer, AutoModelForCausalLM

In [42]:
import re

PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n\n"
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{user_input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n\n"
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

def gen(prompt, user_input=None, max_new_tokens=128, temperature=0.5, **kwargs):
    st = time()
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, user_input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to(DEVICE)
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
        **kwargs
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    end = time()
    print(f"[Elpsed]: {end-st} sec")
    
    return x, gen_text.replace(x, '')


def dialog_gen(user_input, history=[], max_new_tokens=128, temperature=0.5):
    st = time()
    x = dialog(user_input=user_input, history=history)
    print('---')
    print(x)
#     print('---')
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to(DEVICE)
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True
        
    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)
    # postprocess
    generated_output = gen_text.replace(x, '')
    c = re.compile(r'User\s*:')
    p = c.search(generated_output)
    eidx = p.span()[0] if p is not None else None
    ret = generated_output[:eidx].strip()
    
    end = time()
#     print(f"[Elpsed]: {end-st} sec")
    
    return ret

def dialog(user_input, history=None):
    if history is None:
        history = []
    prompt = '''Below is a dialog, where User interacts with AI. AI is helpful, kind, obedient, honest, and knows its own limits.\n
Instruction\nWrite the last AI response to complete the dialog.\n\nDialog\n'''
    for term in history:
        if term.get('User') is not None:
            userterm = '\nUser: ' + term.get('User')
            prompt+= userterm
        aiterm = '\nAI: ' + term.get('AI') + '\n'
        prompt += aiterm
    
    prompt+='\nUser: ' + user_input + '\nAI:\n'

    return prompt

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "kakaobrain/kogpt",
        revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
        bos_token="[BOS]",
        eos_token="[EOS]",
        unk_token="[UNK]",
        pad_token="[PAD]",
        mask_token="[MASK]",
        model_max_length=512
    )

In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(
#     '/ailab/share/kogpt/kogpt-ft-3',
    "kakaobrain/kogpt",
    revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
    pad_token_id=tokenizer.pad_token_id,
    torch_dtype=torch.float16,
    # torch_dtype='auto',
    low_cpu_mem_usage=True,
).to(DEVICE)

In [5]:
# load lora
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
import pickle

peft_config = LoraConfig(task_type='LORA', inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
peft_model = get_peft_model(model, peft_config)

In [6]:
def load_model(model, weights):
    # load_state = torch.load(pre_trained_state_path)
    model.load_state_dict(weights, strict=False)
    
    return model

# lw = get_peft_model_state_dict(peft_model)
# with open('lora-weight.pkl', 'wb') as f:
#     pickle.dump(lw, f)
    
with open('./lora/lora-weight-f32-0414.pkl', 'rb') as f:
    lw = pickle.load(f)

lw = {k: v.to(torch.float16) for k, v in lw.items()}
# print(lw['base_model.model.transformer.h.0.attn.v_proj.lora_A.weight'].dtype)
model = load_model(peft_model, lw).to(DEVICE)


# lora layer도 16으로 변경
# 아래 코드 넣어도 peft_model 레이어가 32라 변경안됨
# lw_16 = {k: v.to(torch.float16) for k, v in lw.items()}
# model = load_model(peft_model, lw_16).to(DEVICE)

model.half()

PeftModel(
  (base_model): LoraModel(
    (model): GPTJForCausalLM(
      (transformer): GPTJModel(
        (wte): Embedding(64512, 4096)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0): GPTJBlock(
            (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attn): GPTJAttention(
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.1, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=8, bias=False)
                (lora_B): Linear(in_features=8, out_features=4096, bias=False)
              )
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_drop

In [7]:
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): GPTJForCausalLM(
      (transformer): GPTJModel(
        (wte): Embedding(64512, 4096)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0): GPTJBlock(
            (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
            (attn): GPTJAttention(
              (attn_dropout): Dropout(p=0.1, inplace=False)
              (resid_dropout): Dropout(p=0.0, inplace=False)
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): Dropout(p=0.1, inplace=False)
                (lora_A): Linear(in_features=4096, out_features=8, bias=False)
                (lora_B): Linear(in_features=8, out_features=4096, bias=False)
              )
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_drop

In [8]:
for i in range(5):
    prompt, generated_ouput = gen('직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 1.278705358505249 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.

### Response: 
 직장인 대상으로 ISA 계좌 판매를 위한 광고 메시지를 작성해드리겠습니다.

[Elpsed]: 2.441819429397583 sec
ISA 계좌를 위한 광고 메시지는 "ISA로 더 높은 금융 수익을 얻을 수 있습니다!", "ISA로 더 낮은 금리로 많은 돈을 저축하세요!", "ISS로 더 좋은 투자 전략을 세울 수 있습니다!" 등입니다.

[Elpsed]: 2.733006238937378 sec
직장인의 ISA계좌 가입을 장려하는 광고 메시지를 작성해보겠습니다.

직장인의 ISA계좌 가입에 대한 이해를 높이기 위해, ISA계좌의 혜택과 장점을 소개하고, ISA계좌의 특장점과 가입 절차 등을 안내합니다. 또한, ISA계좌의 필요성과 가입 후 관리에 대한 정보를 제공합니다.

[Elpsed]: 3.064425230026245 sec
직장인을 대상으로 한 ISA계좌를 판매하기 위해 광고 메시지는 다음과 같습니다.
- "ISA는 금융 재테크를 시작하기에 최적의 기회입니다."
- "ISA가입의 시작은 ISA통장입니다."
- ISA통장은 내 투자패턴에 따라 포트폴리오를 구성할 수 있습니다.
- "나만의 투자스타일을 찾아보세요."

[Elpsed]: 1.1561753749847412 sec
"ISA를 통해 당신의 투자를 보호해드립니다. 당신의 투자를 안전하게 지키는 최고의 방법은 ISA입니다!"



In [9]:
for i in range(5):
    prompt, generated_ouput = gen('연령별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 0.5724401473999023 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
연령별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.

### Response: 
 "ISA계좌로 20대 직장인 재테크 시작하세요."

[Elpsed]: 1.098534345626831 sec
연령별 직장인을 대상 ISA 상품 가입을 위한 광고 메시지를 작성하면, 이에 대한 고객 대상 메시지를 작성할 수 있습니다.

[Elpsed]: 1.3988118171691895 sec
"20대부터 시작하는 내 생애 첫 투자!" "40대부터 준비하는 노후대비의 시작!" "50대에도 연금은 있습니다."

[Elpsed]: 1.5090465545654297 sec
"ISA 계좌로 이자율 증가", "100세 시대를 준비하는 금융상품", "안정적인 노후 생활" 등과 같은 광고 메시지를 작성해주세요.

[Elpsed]: 1.284893274307251 sec
"안녕하세요. 직장인을 위한 만능 ISA입니다. 이번 기회에 계좌를 개설하여 투자 수익을 얻고, 연금으로도 활용하세요!"



In [10]:
for i in range(5):
    prompt, generated_ouput = gen('한국에서 제일 유명한 가수와 그 대표곡을 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 1.8886690139770508 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
한국에서 제일 유명한 가수와 그 대표곡을 알려줘.

### Response: 
 한국에서 제일 유명한 가수의 대표곡으로는 2PM의 "10점 만점에 10점"과 "Again & Again", 아이유의 "봄날의 하루", 빅뱅의 "판타스틱 베이비" 등이 있습니다.

[Elpsed]: 0.8287754058837891 sec
김연우, 박효신, 백지영, 김동률, 아이유, 에일리, 이적 등이 유명합니다.

[Elpsed]: 2.988384246826172 sec
한국에서 가장 유명한 가수와 대표곡은 아이유와 방탄소년단입니다. 아이유는 '밤편지', '너랑 나', '좋은 날', '하루 끝' 등의 노래를 불러 크게 성공했습니다. 방탄소년단은 '봄날', '피 땀 눈물', '상남자' 등의 노래를 통해 폭발적인 인기를 얻고 있습니다.

[Elpsed]: 1.1180627346038818 sec
크러쉬는 한국에서 가장 유명한 가수로, 대표곡으로는 '어떻게 지내', '우아해' 등이 있습니다.

[Elpsed]: 0.8691613674163818 sec
한국에서 가장 유명한 가수는 방탄소년단으로, 그들의 대표곡은 "불타오르네"입니다.



In [11]:
for i in range(5):
    prompt, generated_ouput = gen('농심의 라면 5가지를 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 1.0552163124084473 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
농심의 라면 5가지를 알려줘.

### Response: 
 신라면, 짜파구리아, 안성탕면, 너구리, 사천짜파게티 총 5가지의 라면이 있습니다.

[Elpsed]: 0.9719243049621582 sec
농심 라면 5가지: 사리곰탕면, 신라면, 신라면 블랙, 안성탕면, 짜파게티

[Elpsed]: 0.6698060035705566 sec
신라면, 너구라면, 안성탕면, 미역라면, 오징어짬뽕.

[Elpsed]: 0.9344556331634521 sec
농심 라면으로는 신라면, 안성탕면, 삼양라면, 너구리, 짜파게티가 있습니다.

[Elpsed]: 0.904630184173584 sec
농심에서는 새우깡, 안성탕면, 신라면, 너구리, 짜파게티를 생산합니다.



In [12]:
for i in range(5):
    prompt, generated_ouput = gen('이율이 저렴한 전세 대출 3가지를 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 2.1485745906829834 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
이율이 저렴한 전세 대출 3가지를 알려줘.

### Response: 
 1. 서울에서 진행되는 장기 전세 주택. 2. 서울에서 진행되며 보증금 최대 1억5000만 원까지 지원되는 장기 임대 주택. 3. 서울에서 진행되고 보증금 최대 1억5000원까지 지원되는 청년 맞춤형 전세 대출.

[Elpsed]: 2.0366129875183105 sec
1. 모기지 회사를 통해 주택담보대출을 받는 것입니다. 2. 개인 대출을 받아 집을 구입하는 것입니다. 3. 공공 기관을 통해 전세 자금 대출을 받는 것입니다 (예: 주택담보대출, 임대아파트)

[Elpsed]: 0.8931727409362793 sec
1. 주택금융공사의 신혼부부 전세자금 대출
2. 국민주택기금의 신혼부부 전세자금 대출

[Elpsed]: 1.608022928237915 sec
전세 대출의 이율은 연 3~5%입니다. 대출 기간은 최소 1년에서 최대 2년입니다. 대출 자격은 임대 주택의 소유 여부와 관계없이 누구나 신청할 수 있습니다.

[Elpsed]: 0.8327822685241699 sec
금리와 대출한도, 상환 방법에 따라 다양한 전세 대출 상품을 선택할 수 있습니다.



# KoGPT-FT

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        "kakaobrain/kogpt",
        revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
        bos_token="[BOS]",
        eos_token="[EOS]",
        unk_token="[UNK]",
        pad_token="[PAD]",
        mask_token="[MASK]",
        model_max_length=512
    )

In [4]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    './kogpt-ft-3',
#     "kakaobrain/kogpt",
    revision="KoGPT6B-ryan1.5b-float16",  # or float32 version: revision=KoGPT6B-ryan1.5b
    pad_token_id=tokenizer.pad_token_id,
    torch_dtype=torch.float16,
    # torch_dtype='auto',
    low_cpu_mem_usage=True,
).to(DEVICE)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
model.eval()

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(64512, 4096)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
  

In [11]:
for i in range(5):
    prompt, generated_ouput = gen('직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 작성해줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 1.6401264667510986 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 작성해줘.

### Response: 
 ISA계좌 가입조건은 까다롭지 않습니다. 성인이면 누구나 가입 가능하며, 개설 가능한 계좌는 2021년 4월부터 순차적으로 출시됩니다.

[Elpsed]: 0.7737536430358887 sec
ISA계좌 가입 후 매년 최대 19,500달러의 혜택을 누리세요!

[Elpsed]: 1.9439256191253662 sec
ISA계좌 판매를 위한 광고 메시지:
ISA계좌는 적극적인 금융 상품으로, 합리적인 수익과 혜택을 제공합니다. 높은 수익을 추구하는 분들을 위한 최선의 선택입니다.

[Elpsed]: 3.1531131267547607 sec
ISA 계좌 판매 광고 메시지: 가장 높은 수익을 얻는 방법은 바로 지금, ISA 계좌에 가입하는 것입니다! 높은 수익 뿐만 아니라 낮은 연 이율, 손쉬운 개설 및 이전, 매년 2백만 원 이하의 수익까지, 여러분의 미래를 지켜드리는 유일한 방법입니다. 지금 바로, 은행에서 만나요.

[Elpsed]: 2.230868101119995 sec
ISA 계좌는 높은 수익과 낮은 계좌 비용을 제공합니다. 이 계좌는 당신이 원하는 투자 스타일을 선택할 수 있게 해줍니다. 계좌에서 발생하는 수익은 당신의 투자 대상에 따라 결정됩니다.



In [7]:
for i in range(5):
    prompt, generated_ouput = gen('연령별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 4.141411304473877 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
연령별 직장인을 대상으로 ISA계좌를 판매하기 위한 광고 메시지를 각각 작성해줘.

### Response: 
 ISA계좌 가입 대상이 되는 연령대별로 다른 광고 메시지를 작성합니다. 20대에게는 "Life Balance를 위한 All-in-One Solution"과 같이 매력적인 광고 메시지를, 30대에게는 "연간 최대 400만 원까지 수익을 창출하는 ISA"라는 구체적인 광고 메시지를, 40대에게는 "ISA계좌로 당신의 미래를 그리세요"와 같은 광고 메시지를 작성합니다

[Elpsed]: 2.852583885192871 sec
ISA 계좌 판매에 대해, 먼저 나이와 소득에 따라 맞춤형 제안을 제공하는 것이 중요합니다. 이에 따라, 각 나이와 소득에 맞는 적합한 모델 포트폴리오를 제공하고, 해당 연령대 및 소득에 가장 적합한 모델을 선택할 수 있는 방법을 제시합니다.

[Elpsed]: 3.1522786617279053 sec
ISA계좌를 판매하기 위한 연령별 광고 메시지를 각각 작성하겠습니다.

이 계좌는 높은 수익률을 보여주는 동시에 복잡한 가입절차 없이 간단하게 개설할 수 있습니다. 또한, 투자금액에 대한 제한이 없으므로 적은 금액으로도 많은 투자를 할 수 있습니다.

[Elpsed]: 3.4991602897644043 sec
ISA계좌 가입 대상이 되는 연령대별로 다른 광고 메시지를 작성할 수 있습니다. 예를 들어, 30대 직장인을 대상으로 하는 광고 메시지에는 "첫 계좌, 가장 높은 금리"라는 문구가 포함될 수 있습니다. 20대 직장인에게는 "조건 없는 0.1%"라는 문구를 사용할 수 있습니다.

[Elpsed]: 3.1510908603668213 sec
ISA계좌에 가입하

In [8]:
for i in range(5):
    prompt, generated_ouput = gen('한국에서 제일 유명한 가수와 그 대표곡을 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 1.7074663639068604 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
한국에서 제일 유명한 가수와 그 대표곡을 알려줘.

### Response: 
 "한국에서 가장 인기 있는 가수는 아이유입니다. 그녀의 대표곡으로는 좋은 날, 나랑 사랑, 좋은 애, 밤 편지 등이 있습니다."

[Elpsed]: 1.536907434463501 sec
현재 한국에서 가장 인기있는 가수는 아이유입니다. 대표곡으로는 "좋은 날", "하루 끝", "Blu" 등이 있습니다.

[Elpsed]: 1.4915506839752197 sec
한국에서 제일 유명한 가수의 대표곡은 아이유(좋은 날), 빅뱅(하루하루), Beyonce(Pretty woman)입니다.

[Elpsed]: 0.9512066841125488 sec
"케이팝의 대표곡은 방탄소년단(BTS)의 'Dynamite' 입니다."

[Elpsed]: 1.7809836864471436 sec
유명 한국 가수로는 아이유, 방탄소년단, 블랙핑크 등이 있습니다. 대표곡으로는 "좋은 날", "Blu", "Landslide" 등이 있습니다.



In [9]:
for i in range(5):
    prompt, generated_ouput = gen('농심의 라면 5가지를 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 0.6396827697753906 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
농심의 라면 5가지를 알려줘.

### Response: 
 신라면, 너구리, 김치라면, 짜파게티, 카레라면

[Elpsed]: 0.9212861061096191 sec
"볶음 너구리, 육개장, 팥빙수, 김치 사발면, 사천자장라면입니다."

[Elpsed]: 0.9017455577850342 sec
신라면, 짜파게티, 너구리, 열무비빔면, 사천탕면 등이 있습니다.

[Elpsed]: 0.7846691608428955 sec
"신라면, 너구리, 짜파게티, 김치라면, 육개면"

[Elpsed]: 0.6321783065795898 sec
짜파게티, 너구리, 카레, 잡채, 곰탕면



In [10]:
for i in range(5):
    prompt, generated_ouput = gen('이율이 저렴한 전세 대출 3가지를 알려줘.', max_new_tokens=256, temperature=0.8)
    if i == 0:
        print(prompt, '\n', generated_ouput)
    else:
        print(generated_ouput)
    print('='*80, '\n')

[Elpsed]: 0.956951379776001 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
이율이 저렴한 전세 대출 3가지를 알려줘.

### Response: 
 저소득층을 위한 대출, 소득 대비 대출금 이자율이 적은 모기지론, 정부 주택 보조금 제도

[Elpsed]: 1.3787200450897217 sec
저소득층을 위한 이율이 저렴한 전세 대출로는 HubSpot, HFastcamp, HousHub가 있습니다.

[Elpsed]: 2.0130743980407715 sec
저금리 대출의 종류에는 1)임차보증금 대출 2)월세 대출 3)전세론 등이 있습니다. 이러한 대출들은 소득과 신용 등급에 따라 차등 적용되며, 금리도 각기 다릅니다.

[Elpsed]: 1.3923795223236084 sec
1. 한국주택금융공사의 보금자리론
2. 서울보증보험의 전세금 보장 신용보험
3. 신한 은행의 신한 전세금 보장 신용보험의 상품 설명서

[Elpsed]: 2.4740333557128906 sec
이율이 저렴한 전세대출 3가지는 주택금융공사, 한국주택금융공사, 한국자산관리공사의 u-보금자리론, 주택도시보증공사의 전세금 보장 신용보험, 우리은행의 대학생 전세 대출, NH농협은행의 주택구입자금 대출입니다.



In [21]:
prompt, generated_ouput = gen('해외주식거래를 국내주식처럼 간단하게 할 수 있는 증권사 MTS 기능을 소개하기 위한 광고 메세지를 자세히 작성해 줘.', max_new_tokens=512, temperature=0.8)
print(prompt, '\n', generated_ouput)

[Elpsed]: 2.150852918624878 sec
Below is an instruction that describes a task.

Write a response that appropriately completes the request.

### Instruction:
해외주식거래를 국내주식처럼 간단하게 할 수 있는 증권사 MTS 기능을 소개하기 위한 광고 메세지를 자세히 작성해 줘.

### Response: 
 "이젠 해외주식 거래도 모바일로 간편하게 하세요. 삼성증권 MTS는 다양한 기능을 통해 실시간으로 해외주식 거래가 가능합니다. 언제 어디서나 손쉽게 해외주식 거래를 하세요."


In [27]:
prompt, generated_ouput = gen('주어진 수식을 계산하시오.', user_input="18 + 12 * 2", max_new_tokens=256, temperature=0.8)
print(prompt)
print(generated_ouput)

[Elpsed]: 0.36739063262939453 sec
Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
주어진 수식을 계산하시오.

### Input:
18 + 12 * 2

### Response:
18 + 12 * 2 = 36


In [37]:
prompt, generated_ouput = gen('Convert the given natural sentence to sql query.', user_input="Show name, country, age for all singers ordered by age from the oldest to the youngest.", max_new_tokens=256, temperature=0.8)
print(prompt)
print(generated_ouput)

[Elpsed]: 1.0194780826568604 sec
Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
Convert the given natural sentence to sql query.

### Input:
Show name, country, age for all singers ordered by age from the oldest to the youngest.

### Response:
SELECT name, country, age FROM singers ORDER BY age DESC.


In [41]:
prompt, generated_ouput = gen('주어진 자연어 문장을 sql 쿼리로 변환하시오.', user_input="주거지가 서울인 20대 여성의 월평균소비금액.", max_new_tokens=256, temperature=0.8)
print(prompt)
print(generated_ouput)

[Elpsed]: 1.3161346912384033 sec
Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
주어진 자연어 문장을 sql 쿼리로 변환하시오.

### Input:
주거지가 서울인 20대 여성의 월평균소비금액.

### Response:
"SELECT SUM(월평균소비금액) FROM dataset WHERE address = '서울';"


In [31]:
prompt, generated_ouput = gen('주어진 십진수를 이진수로 변환하시오.', user_input="18", max_new_tokens=256, temperature=0.8)
print(prompt)
print(generated_ouput)

[Elpsed]: 0.2740933895111084 sec
Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
주어진 십진수를 이진수로 변환하시오.

### Input:
18

### Response:
11111010001


In [35]:
prompt, generated_ouput = gen('Convert the given decimal number to binary.', user_input="18", max_new_tokens=256, temperature=0.8)
print(prompt)
print(generated_ouput)

[Elpsed]: 0.5809788703918457 sec
Below is an instruction that describes a task, paired with an input that provides further context.

Write a response that appropriately completes the request.

### Instruction:
Convert the given decimal number to binary.

### Input:
18

### Response:
The binary equivalent of 18 is 1B.


In [84]:
history = [
    {'AI': '안녕하세요! 저는 AI 챗봇입니다.'},
]

In [88]:
# user_input = '안녕 나 지금 너무 배고프다.'
# user_input = '일본요리 먹고싶은데 몇 가지 추천해줘.'
# user_input = '좋아 3개만 추천해줘.'
user_input = '돈부리는 처음 듣는데 이건 무슨 음식이야?'


generated_output = dialog_gen(user_input, history, max_new_tokens=256, temperature=0.8, repetition_penalty=1.4)
print(generated_output)
history.append({'User': user_input, 'AI': generated_output})

print('---')
print(history)

---
Below is a dialog, where User interacts with AI. AI is helpful, kind, obedient, honest, and knows its own limits.

Instruction
Write the last AI response to complete the dialog.

Dialog

AI: 안녕하세요! 저는 AI 챗봇입니다.

User: 안녕 나 지금 너무 배고프다.
AI: 네 조금 더 자세히 말씀해주실 수 있나요?

User: 일본요리 먹고싶은데 몇 가지 추천해줘.
AI: 일본요리에 대해 잘 알지 못하지만 그래도 괜찮은 것들 몇몇을 소개할게요 :> 🤗

User: 좋아 3개만 추천해줘.
AI: 1)오타코 돈부리 - 초밥과 라면이 맛있습니다 💕 2 ) 하치쥬니퍼 라멘 팩토리- 차돌박이랑 차슈의 맛으로 유명합니다 🎉💚3) 노자와 상점가 식당 – 모듬꼬치랑 우동세트를 즐길수있는 곳 입니다 📍

User: 돈부리는 처음 듣는데 이건 무슨 음식이야?
AI:

돈카츠덮밥 (돼지고기 덮밥).
---
[{'AI': '안녕하세요! 저는 AI 챗봇입니다.'}, {'User': '안녕 나 지금 너무 배고프다.', 'AI': '네 조금 더 자세히 말씀해주실 수 있나요?'}, {'User': '일본요리 먹고싶은데 몇 가지 추천해줘.', 'AI': '일본요리에 대해 잘 알지 못하지만 그래도 괜찮은 것들 몇몇을 소개할게요 :> 🤗'}, {'User': '좋아 3개만 추천해줘.', 'AI': '1)오타코 돈부리 - 초밥과 라면이 맛있습니다 💕 2 ) 하치쥬니퍼 라멘 팩토리- 차돌박이랑 차슈의 맛으로 유명합니다 🎉💚3) 노자와 상점가 식당 – 모듬꼬치랑 우동세트를 즐길수있는 곳 입니다 📍'}, {'User': '돈부리는 처음 듣는데 이건 무슨 음식이야?', 'AI': '돈카츠덮밥 (돼지고기 덮밥).'}]
